# 🔧 Lab 02: Agent s Nástrojmi

**Cieľ:** Naučiť sa vytvárať a používať nástroje (tools) pre AI agenta.

**Čas:** 60 minút

---

## Čo sa naučíš:
- Ako definovať vlastné nástroje
- Ako agent vyberá správny nástroj
- Best practices pre návrh nástrojov

---

In [ ]:
#@title 🔧 Setup
!pip install -q google-generativeai smolagents litellm

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from smolagents import CodeAgent, tool
from smolagents.models import LiteLLMModel

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    api_key=os.environ["GOOGLE_API_KEY"]
)

print("✅ Setup OK")

## Časť 1: Čo je Tool?

**Tool (nástroj)** je funkcia ktorú agent môže zavolať.

Príklady:
- 🌤️ Získať počasie
- 🔍 Vyhľadať v databáze
- 📧 Poslať email
- 🧮 Vypočítať hodnotu

### Anatómia nástroja:
```python
@tool
def my_tool(param1: str, param2: int) -> str:
    """
    Popis čo nástroj robí.  <-- Agent číta tento popis!
    
    Args:
        param1: Popis parametra 1
        param2: Popis parametra 2
    """
    # Logika
    return "výsledok"
```

In [ ]:
#@title 🔧 Prvý nástroj: Kalkulačka

@tool
def calculate(expression: str) -> str:
    """
    Vypočíta matematický výraz.
    
    Args:
        expression: Matematický výraz na výpočet (napr. "2 + 2 * 3")
    """
    try:
        result = eval(expression)
        return f"Výsledok: {result}"
    except Exception as e:
        return f"Chyba: {str(e)}"

# Test nástroja priamo
print(calculate("15 * 7 + 23"))

In [ ]:
#@title 🤖 Agent s kalkulačkou

calc_agent = CodeAgent(
    tools=[calculate],
    model=model,
    max_steps=3
)

# Agent teraz VIE počítať presne
result = calc_agent.run("Koľko je 15% z 1240?")
print(result)

## Časť 2: Viac nástrojov

In [ ]:
#@title 🔧 Definícia viacerých nástrojov

@tool
def get_weather(city: str) -> str:
    """
    Získa aktuálne počasie pre dané mesto.
    
    Args:
        city: Názov mesta (napr. "Praha", "Brno", "Bratislava")
    """
    # Demo databáza (v reále by si volal weather API)
    weather_data = {
        "praha": {"temp": 12, "condition": "☀️ jasno"},
        "brno": {"temp": 10, "condition": "🌤️ polojasno"},
        "bratislava": {"temp": 14, "condition": "⛅ oblačno"},
        "vienna": {"temp": 11, "condition": "🌧️ dážď"},
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        return f"{city}: {data['temp']}°C, {data['condition']}"
    return f"Počasie pre {city} nie je dostupné"

@tool
def get_exchange_rate(from_currency: str, to_currency: str) -> str:
    """
    Získa výmenný kurz medzi menami.
    
    Args:
        from_currency: Zdrojová mena (napr. "EUR", "USD", "CZK")
        to_currency: Cieľová mena
    """
    # Demo kurzy
    rates = {
        ("EUR", "CZK"): 25.2,
        ("EUR", "USD"): 1.08,
        ("USD", "CZK"): 23.3,
        ("CZK", "EUR"): 0.04,
    }
    
    key = (from_currency.upper(), to_currency.upper())
    if key in rates:
        return f"1 {from_currency.upper()} = {rates[key]} {to_currency.upper()}"
    return f"Kurz {from_currency}/{to_currency} nie je dostupný"

print("✅ Nástroje definované:")
print("   - calculate")
print("   - get_weather")
print("   - get_exchange_rate")

In [ ]:
#@title 🤖 Multi-tool Agent

multi_agent = CodeAgent(
    tools=[calculate, get_weather, get_exchange_rate],
    model=model,
    max_steps=5,
    verbosity_level=1  # Ukáže ktoré nástroje použil
)

print("Agent má prístup k:")
for t in multi_agent.tools.values():
    print(f"   - {t.name}")
print()

In [ ]:
#@title ▶️ Test: Komplexná úloha

task = """
Plánujem výlet do Prahy. Povedz mi:
1. Aké je tam počasie?
2. Koľko CZK dostanem za 100 EUR?
"""

print(f"📋 Úloha:\n{task}")
print("="*50)

result = multi_agent.run(task)

print("="*50)
print(f"\n✅ Odpoveď:\n{result}")

## Časť 3: Nástroj s logikou

In [ ]:
#@title 🔧 Komplexnejší nástroj: Product Search

import json

# Simulovaná produktová databáza
PRODUCTS = [
    {"id": 1, "name": "Laptop Dell XPS 15", "price": 1299, "category": "electronics", "stock": 5},
    {"id": 2, "name": "iPhone 15 Pro", "price": 999, "category": "electronics", "stock": 12},
    {"id": 3, "name": "Sony WH-1000XM5", "price": 349, "category": "electronics", "stock": 8},
    {"id": 4, "name": "Office Chair Ergonomic", "price": 299, "category": "furniture", "stock": 3},
    {"id": 5, "name": "Standing Desk", "price": 449, "category": "furniture", "stock": 0},
]

@tool
def search_products(query: str, max_price: int = 10000) -> str:
    """
    Vyhľadá produkty v katalógu.
    
    Args:
        query: Hľadaný výraz (časť názvu alebo kategória)
        max_price: Maximálna cena v EUR (predvolené: 10000)
    """
    results = []
    query_lower = query.lower()
    
    for product in PRODUCTS:
        name_match = query_lower in product["name"].lower()
        category_match = query_lower in product["category"].lower()
        price_ok = product["price"] <= max_price
        
        if (name_match or category_match) and price_ok:
            results.append(product)
    
    if not results:
        return "Žiadne produkty nenájdené."
    
    output = f"Nájdených {len(results)} produktov:\n"
    for p in results:
        stock_status = "✅ na sklade" if p["stock"] > 0 else "❌ vypredané"
        output += f"- {p['name']}: {p['price']}€ ({stock_status})\n"
    
    return output

@tool
def check_stock(product_name: str) -> str:
    """
    Skontroluje dostupnosť konkrétneho produktu.
    
    Args:
        product_name: Názov produktu (nemusí byť presný)
    """
    for product in PRODUCTS:
        if product_name.lower() in product["name"].lower():
            if product["stock"] > 0:
                return f"{product['name']}: {product['stock']} kusov na sklade"
            else:
                return f"{product['name']}: VYPREDANÉ"
    return f"Produkt '{product_name}' nenájdený"

# Test
print(search_products("electronics", 500))

In [ ]:
#@title 🛒 E-commerce Agent

shop_agent = CodeAgent(
    tools=[search_products, check_stock, calculate],
    model=model,
    max_steps=5,
    system_prompt="""Si asistent e-shopu. Pomáhaš zákazníkom nájsť produkty.
    Vždy:
    - Odpovedaj priateľsky
    - Uvádzaj ceny
    - Informuj o dostupnosti
    """
)

result = shop_agent.run("Hľadám slúchadlá do 400€. Čo máte?")
print(result)

In [ ]:
#@title ▶️ Test: Komplexný scenár

result = shop_agent.run("""
Potrebujem vybaviť home office. 
Rozpočet mám 800€.
Čo by si mi odporučil?
""")
print(result)

## Časť 4: Best Practices

### ✅ Dobré nástroje:
1. **Jasný popis** - agent musí pochopiť čo nástroj robí
2. **Špecifické parametre** - s typmi a popismi
3. **Zmysluplný output** - štruktúrovaná odpoveď
4. **Error handling** - nástroj nepadne

### ❌ Zlé nástroje:
1. Vágny popis
2. Príliš veľa parametrov
3. Nástroj robí príliš veľa vecí
4. Nevratí užitočnú informáciu

In [ ]:
#@title ❌ Príklad ZLÉHO nástroja

# ❌ Zlý nástroj - nejasný popis, chýbajú typy
@tool
def do_stuff(x):
    """Does stuff with x"""
    return str(x * 2)

# Agent nevie kedy a ako tento nástroj použiť!

In [ ]:
#@title ✅ Príklad DOBRÉHO nástroja

@tool
def convert_temperature(value: float, from_unit: str, to_unit: str) -> str:
    """
    Prevedie teplotu medzi jednotkami (Celsius, Fahrenheit, Kelvin).
    
    Args:
        value: Číselná hodnota teploty
        from_unit: Zdrojová jednotka ("C", "F", alebo "K")
        to_unit: Cieľová jednotka ("C", "F", alebo "K")
    """
    # Najprv prevedieme na Celsius
    if from_unit.upper() == "F":
        celsius = (value - 32) * 5/9
    elif from_unit.upper() == "K":
        celsius = value - 273.15
    else:
        celsius = value
    
    # Potom na cieľovú jednotku
    if to_unit.upper() == "F":
        result = celsius * 9/5 + 32
    elif to_unit.upper() == "K":
        result = celsius + 273.15
    else:
        result = celsius
    
    return f"{value}°{from_unit.upper()} = {result:.2f}°{to_unit.upper()}"

# Test
print(convert_temperature(100, "C", "F"))

---

## 🏋️ Cvičenie: Vytvor vlastný toolset

**Úloha:** Vytvor agenta pre zákaznícku podporu s nástrojmi:

1. `search_faq(query)` - vyhľadá v FAQ
2. `create_ticket(summary, priority)` - vytvorí support ticket
3. `check_order_status(order_id)` - skontroluje stav objednávky

In [ ]:
#@title ✏️ Tvoje riešenie

# FAQ databáza
FAQ = {
    "refund": "Vrátenie peňazí je možné do 14 dní od doručenia.",
    "shipping": "Štandardné doručenie trvá 3-5 pracovných dní.",
    "payment": "Prijímame karty Visa, Mastercard a PayPal.",
}

# Objednávky
ORDERS = {
    "ORD-001": {"status": "shipped", "tracking": "SK123456"},
    "ORD-002": {"status": "processing"},
    "ORD-003": {"status": "delivered"},
}

# TODO: Implementuj nástroje

@tool
def search_faq(query: str) -> str:
    """
    # TU DOPLŇ DOCSTRING
    """
    # TU DOPLŇ IMPLEMENTÁCIU
    pass

@tool
def create_ticket(summary: str, priority: str) -> str:
    """
    # TU DOPLŇ DOCSTRING
    """
    # TU DOPLŇ IMPLEMENTÁCIU
    pass

@tool
def check_order_status(order_id: str) -> str:
    """
    # TU DOPLŇ DOCSTRING
    """
    # TU DOPLŇ IMPLEMENTÁCIU
    pass


# Vytvor agenta
support_agent = CodeAgent(
    tools=[search_faq, create_ticket, check_order_status],
    model=model,
    max_steps=5
)

# Test
# result = support_agent.run("Kde je moja objednávka ORD-001?")
# print(result)

---

## 📝 Reflexia

1. **Prečo je dôležitý dobrý docstring?**

2. **Ako agent rozhoduje ktorý nástroj použiť?**

3. **Kedy má zmysel rozdeliť funkcionalitu do viacerých nástrojov?**

---

## ➡️ Ďalší krok

V ďalšom labe sa naučíme **RAG (Retrieval-Augmented Generation)** - ako dať agentovi prístup k dokumentom.

→ **[03_RAG_Agent.ipynb](03_RAG_Agent.ipynb)**